In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks'
df = pd.read_csv('data/multinli.train.koNLI.tsv', delimiter = '\t', quoting = 3)
df = df.dropna()

# gold_label을 수치화한다. [contradiction (모순 관계) = 0, entailment (얽힘) = 1, neutral (중립) = 2]
df['label'] = LabelEncoder().fit_transform(df['gold_label'])

df = df[:10000]   # 시험용으로 조금만 사용
df.head()

/content/drive/MyDrive/Colab Notebooks


,sentence1,sentence2,gold_label,label
0,개념적으로 크림 스키밍은 제품과 지리라는 두 가지 기본 차원을 가지고 있다.,제품과 지리학은 크림 스키밍을 작동시키는 것이다.,neutral,2
1,시즌 중에 알고 있는 거 알아? 네 레벨에서 다음 레벨로 잃어버리는 거야 브레이브스...,사람들이 기억하면 다음 수준으로 물건을 잃는다.,entailment,1
2,우리 번호 중 하나가 당신의 지시를 세밀하게 수행할 것이다.,우리 팀의 일원이 당신의 명령을 엄청나게 정확하게 실행할 것이다.,entailment,1
3,어떻게 아세요? 이 모든 것이 다시 그들의 정보다.,이 정보는 그들의 것이다.,entailment,1
4,"그래, 만약 네가 테니스화 몇 개를 사러 간다면, 나는 왜 그들이 100달러대에서 ...",테니스화의 가격은 다양하다.,neutral,2


In [ ]:
df['label'] = LabelEncoder().fit_transform(df['gold_label'])

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", cache_dir='bert_ckpt', do_lower_case=False)

In [ ]:
# Bert Tokenizer
# 참조: https://huggingface.co/transformers/main_classes/tokenizer.html?highlight=encode_plus#transformers.PreTrainedTokenizer.encode_plus
def bert_tokenizer_v2(sent1, sent2):
    
    encoded_dict = tokenizer.encode_plus(
        text = sent1,
        text_pair = sent2,
        add_special_tokens = True,      # Add '[CLS]' and '[SEP]'
        max_length = MAX_LEN,           # Pad & truncate all sentences.
        pad_to_max_length = True,
        return_attention_mask = True    # Construct attn. masks.
    )
    
    input_id = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask'] # And its attention mask (simply differentiates padding from non-padding).
    token_type_id = encoded_dict['token_type_ids']  # differentiate two sentences
    
    return input_id, attention_mask, token_type_id

In [ ]:
def build_data(sent1, sent2):
    x_ids = []
    x_msk = []
    x_typ = []

    for s1, s2 in tqdm(zip(sent1, sent2)):
        input_id, attention_mask, token_type_id = bert_tokenizer_v2(s1, s2)
        x_ids.append(input_id)
        x_msk.append(attention_mask)
        x_typ.append(token_type_id)

    x_ids = np.array(x_ids, dtype=int)
    x_msk = np.array(x_msk, dtype=int)
    x_typ = np.array(x_typ, dtype=int)

    return x_ids, x_msk, x_typ

In [ ]:
sent1 = df['sentence1']
sent2 = df['sentence2']
y_label = df['label']
MAX_LEN = 24 * 2

x_train1, x_test1, x_train2, x_test2, y_train, y_test = train_test_split(sent1, sent2, y_label, test_size=0.1, random_state = 0)

x_train_ids, x_train_msk, x_train_typ = build_data(x_train1, x_train2)
x_test_ids, x_test_msk, x_test_typ = build_data(x_test1, x_test2)

y_train = np.array(y_train).reshape(-1, 1)
y_test = np.array(y_test).reshape(-1, 1)

x_train_ids.shape, y_train.shape, x_test_ids.shape, y_test.shape

((9000, 48), (9000, 1), (1000, 48), (1000, 1))

In [ ]:
x_train_ids[1]

array([   101,   9011,    117,   9711,  10622, 103995,   9246, 119266,
        12965,  48549,    102,   9011,    117,   9711,  10622,   9246,
       119284,   9460,   9647, 119138,  12965,  48549,    119,    102,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0])

In [ ]:
x_train_msk[1]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0])

In [ ]:
x_train_typ[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0])

In [ ]:
word2idx = tokenizer.vocab
idx2word = {v:k for k, v in word2idx.items()}
print([idx2word[x] for x in x_train_ids[1]])

['[CLS]', '네', ',', '집', '##을', '완전히', '마', '##쳤', '##어', '##요', '[SEP]', '네', ',', '집', '##을', '마', '##칠', '수', '있', '##었', '##어', '##요', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


In [ ]:
bert_model = TFBertModel.from_pretrained("bert-base-multilingual-cased", cache_dir='bert_ckpt')
bert_model.summary() # bert_model을 확인한다. trainable params = 177,854,978

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "tf_bert_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  177853440 
Total params: 177,853,440
Trainable params: 177,853,440
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# TFBertMainLayer는 fine-tuning을 하지 않는다. (시간이 오래 걸림)
bert_model.trainable = False
bert_model.summary() # bert_model을 다시 확인한다. trainable params = 0

Model: "tf_bert_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  177853440 
Total params: 177,853,440
Trainable params: 0
Non-trainable params: 177,853,440
_________________________________________________________________


In [ ]:
# BERT 입력
# ---------
x_input_ids = Input(batch_shape = (None, MAX_LEN), dtype = tf.int32)
x_input_msk = Input(batch_shape = (None, MAX_LEN), dtype = tf.int32)
x_input_typ = Input(batch_shape = (None, MAX_LEN), dtype = tf.int32)

# BERT 출력
# ---------
output_bert = bert_model([x_input_ids, x_input_msk, x_input_typ])[1]

# Downstream task : 자언어 추론 (NLI)
# ----------------------------------
y_output = Dense(3, activation = 'softmax')(output_bert)
model = Model([x_input_ids, x_input_msk, x_input_typ], y_output)
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = Adam(learning_rate = 0.001))
model.summary()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [ ]:
x_train = [x_train_ids, x_train_msk, x_train_typ]
x_test = [x_test_ids, x_test_msk, x_test_typ]
hist = model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs=1, batch_size=1024)

# Epoch 1/3
# 157/157 [==============================] - ETA: 0s - loss: 0.6245WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
# WARNING:tensorflow:The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
# 157/157 [==============================] - 936s 6s/step - loss: 0.6245 - val_loss: 0.6070
# Epoch 2/3
# 157/157 [==============================] - 926s 6s/step - loss: 0.6184 - val_loss: 0.6023
# Epoch 3/3
# 157/157 [==============================] - 903s 6s/step - loss: 0.6134 - val_loss: 0.5973


9/9 [==============================] - 42s 4s/step - loss: 1.1073 - val_loss: 1.0937


In [ ]:
# Loss history를 그린다
plt.plot(hist.history['loss'], label='Train loss')
plt.plot(hist.history['val_loss'], label = 'Test loss')
plt.legend()
plt.title("Loss history")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.show()

In [ ]:
# 시험 데이터로 학습 성능을 평가한다
pred = model.predict(x_test)
y_pred = np.argmax(pred, axis=1).reshape(-1, 1)
accuracy = (y_pred == y_test).mean()
print("\nAccuracy = %.2f %s" % (accuracy * 100, '%'))


Accuracy = 33.60 %
